In [1]:
import sys
# the mock-0.3.1 dir contains testcase.py, testutils.py & mock.py
from get_data import get_data
from train import train,score
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

Device used = cuda


In [2]:
ns = [10,50,100,500,2000]
augs = [4,8,16,32,64]


Xt, Yt = get_data(type="dev")

length = len(ns)*len(augs)
data = {"n":[0]*length,
       "augs":[0]*length,
       "score":[0.0]*length,
       "vocab":[0]*length}
df = pd.DataFrame(data)

i = 0
for n in ns:
    X,Y = get_data(f"n_{n}",early_return=False)
    model, vocab = train( X, Y, epochs = 20, batch_size=64,embed_dim=100,lstm_dim=100,min_df=0,verbose=False)
    acc = score(model,vocab,Xt,Yt,verbose=False)
    df.at[i,"n"] = n
    df.at[i,"score"] = acc
    df.at[i,"vocab"] = len(vocab)
    i+=1
    for aug in augs:
        #print(f"\neda_augs_{aug}_n_{n}")
        X,Y = get_data(type=f"eda_augs_{aug}_n_{n}")
        model, vocab = train( X, Y, epochs = 20, batch_size=64,embed_dim=100,lstm_dim=100,min_df=0,verbose=False)
        acc = score(model,vocab,Xt,Yt,verbose=False)
        df.at[i,"n"] = n
        df.at[i, "augs"] = aug
        df.at[i,"score"] = acc
        df.at[i,"vocab"] = len(vocab)
        i += 1

In [3]:
df

,n,augs,score,vocab
0,10.0,0.0,0.431,187.0
1,10.0,4.0,0.548,1930.0
2,10.0,8.0,0.523,2267.0
3,10.0,16.0,0.587,2828.0
4,10.0,32.0,0.606,3540.0
5,10.0,64.0,0.573,4425.0
6,50.0,0.0,0.435,987.0
7,50.0,4.0,0.596,5892.0
8,50.0,8.0,0.656,6916.0
9,50.0,16.0,0.679,8177.0


In [4]:
df.to_csv("EDA_results.csv")

NameError: name 'df_acc' is not defined

In [ ]:
g = sns.FacetGrid(df,col="n")
g.map(sns.scatterplot,"augs","score")